<a href="https://colab.research.google.com/github/ThamirisAdriano/gen_ia_mlet/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar as bibliotecas necessárias
!pip install transformers datasets torch

### Importar as bibliotecas necessárias


In [ ]:

from datasets import load_dataset, load_metric
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

### Carregar o conjunto de dados IMDB

In [ ]:
dataset = load_dataset('imdb')

### Reduzir o conjunto de dados para acelerar o treinamento


In [ ]:
small_train_dataset = dataset['train'].select(range(1000))
small_test_dataset = dataset['test'].select(range(1000))

# Carregar o tokenizador e o modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Função para tokenizar os dados

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenizar o conjunto de dados de treino e teste
small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_test_dataset = small_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Selecionar as colunas de entrada e saída


In [ ]:
small_train_dataset = small_train_dataset.rename_column("label", "labels")
small_train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
small_test_dataset = small_test_dataset.rename_column("label", "labels")
small_test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

### Carregar a métrica de acurácia


In [ ]:
accuracy_metric = load_metric("accuracy")

# Função de avaliação para calcular a acurácia
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

### Configurações do treinamento com otimizações para acelerar o processo


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Aumentar o tamanho do lote
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Reduzir para 1 época
    weight_decay=0.01,
    fp16=True,  # Habilitar precisão mista
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Definir o Trainer


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics  # Adicionar a função de métricas
)

### Executar o treinamento (fine-tuning)


In [ ]:
trainer.train()

# Avaliar o modelo ajustado
results = trainer.evaluate()

print(f"Fine-tuned Loss: {results['eval_loss']}")
print(f"Fine-tuned Accuracy: {results['eval_accuracy']}")



Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.003832,1.000000


Fine-tuned Loss: 0.003832147689536214
Fine-tuned Accuracy: 1.0


### Comparar com o modelo original (sem fine-tuning)


In [ ]:
original_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Garantir que o modelo original esteja na GPU (ou na CPU)
original_model.to(training_args.device)

trainer.model = original_model
original_results = trainer.evaluate()

print(f"Original Loss: {original_results['eval_loss']}")
print(f"Original Accuracy: {original_results['eval_accuracy']}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original Loss: 0.7512527108192444
Original Accuracy: 0.257
